I have created project directory for this assignment in my Google drive and I have also added repository to github. I wanted to do this assignment as a python project. All the things are there in code implemented.  [SpotifyPlaylistCompletion](https://github.com/PacificG/SpotifyPlaylistCompletion)

In [ ]:
#@title
path = "/content/drive/MyDrive"

In [ ]:
#@title
cd $path

In [ ]:
#@title
# try:
#     makeerror==1
# except:
#     !wget https://github.com/gravitational/teleconsole/releases/download/0.4.0/teleconsole-v0.4.0-linux-amd64.tar.gz
!tar -xvf teleconsole-v0.4.0-linux-amd64.tar.gz
# makeerror=False
import subprocess as sp
process = sp.Popen("/content/drive/MyDrive/teleconsole",shell=True,stdin=sp.PIPE,stdout=sp.PIPE,stderr=sp.PIPE)
for i in range(6):
    print(process.stdout.readline().decode())
!apt install screen
!screen -d -m bash

# Data

In [ ]:
data_path = "/content/drive/MyDrive/workspace/PlaylistCompletion/spotifydata/testdata"

In [4]:
cd /content/drive/MyDrive/workspace/PlaylistCompletion/

/content/drive/MyDrive/workspace/PlaylistCompletion


In [ ]:
%%capture
#installing necessary libraries if they are not installed
!pip install -r requirements.txt

Let's first look at the dataset

In [ ]:
import datasets
import importlib
importlib.reload(datasets)

<module 'datasets' from '/content/drive/My Drive/workspace/PlaylistCompletion/datasets.py'>

necessary codes are in seperate modules and i am importing them to use them. 

In [ ]:
#code for datasets module 



```python

import os
import json
import pandas as pd
import csv
class Dataset:
    """
    Custom class to work with the dataset
    """
    track_uri2id = {}   #dictonary containing mappings from trackURI to track_id
    track_id2uri = {}   #dictonary containing mappings from track_id to trackURI
    track_id2aauri = {}  #dictonary containing mappings from track_id to album_uri and artist_uri for that track
    albums_uri2id = {}  #dictonary containing mappings from album_uri to album_id
    artists_uri2id = {} #dictonary containing mappings from artist_uri to artist_id
    
    def __init__(self, playlistPath):
        """
        processes all the json files in path provided. for all the playlists in those files. creates mappings necessary 
        for smooth fetching of artists, albums and tracks data
        """
        dirs = os.listdir(playlistPath)
        self.files = {}
        trackid, albumid, artistid = 0,0,0 #ids are integers starting from 0. artists, albums and tracks have seperate ids
        for file in dirs:
            with open(f'{playlistPath}/{file}') as f:
                self.files[file] = json.load(f)
                for playlist in self.files[file]['playlists']:
                    for track in playlist['tracks']:
                        track_uri, album_uri, artist_uri = track['track_uri'], track['album_uri'], track['artist_uri']
                        if track_uri not in self.track_uri2id:
                            trackid += 1
                            self.track_uri2id[track_uri] = trackid
                            self.track_id2uri[trackid] = (track_uri, track['track_name'])
                            if album_uri not in self.albums_uri2id:
                                albumid += 1
                                self.albums_uri2id[album_uri] = (albumid, track['album_name'])
                            
                            if artist_uri not in self.artists_uri2id:
                                artistid += 1
                                self.artists_uri2id[artist_uri] = (artistid, track['artist_name'])

                        self.track_id2aauri[self.track_uri2id[track_uri]] = (album_uri, artist_uri)

    def playlist_gen(self):
        """
        this generator yields playlists one by one
        format of playlist is like this:
        {pid:'101', title:'playlist name', "trakcs":[track_id for tracks in playlist]}
        """
        for file in self.files:
            for playlist in self.files[file]['playlists']:
                this_playlist = {}
                this_playlist['pid'] = playlist['pid']
                this_playlist['title']= playlist["name"]
                this_playlist['tracks'] = [self.track_uri2id[track['track_uri']] for track in playlist['tracks']]

                yield this_playlist

    
    def trackName4tid(self, track_id):
        """returns track Name for given rack id"""
        return self.track_id2uri[track_id][1]

    def albumName4tid(self, track_id):
        """
        returns album name for a given track id
        """
        return self.albums_uri2id[self.track_id2aauri[track_id][0]][1]

    def artistName4tid(self, track_id):
        """
        returns artist name for a given track id
        """
        return self.artists_uri2id[self.track_id2aauri[track_id][1]][1]
                    
    def playlistCSV(self, csvpath):
        """
        helper method to create intermediate csv for playlists
        """
        with open(f'{csvpath}/playlist.csv', 'w', newline='') as f:
            writer = csv.writer(f)
            names = ['name', 'collaborative','num_tracks','num_artists',
                 'num_albums','num_followers','num_edits','modified_at','duration_ms']
            writer.writerow(names)
            for file in self.files:
                for playlist in self.files[file]['playlists']:
                    writer.writerow([playlist[name] for name in names])
    def trackCSV(self, csvpath):
        """
        helper method to create intermediate csv for tracks
        """
        with open(f'{csvpath}/track.csv','w', newline='') as f:
            writer = csv.writer(f)
            names = ['pid', 'pos','track_uri']
            writer.writerow(names)
            for file in self.files:
                for playlist in self.files[file]['playlists']:
                    for track in playlist['tracks']:
                        writer.writerow([playlist['pid'], track['pos'], track['track_uri']])
    
    
```



In [ ]:
d = datasets.Dataset('spotifydata/testdata') #passing the data path

In [ ]:
#let's check our structure of playlists
for playlist in d.playlist_gen():
  print(playlist)

{'pid': 994000, 'title': 'funky', 'tracks': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56]}
{'pid': 994001, 'title': 'Clean', 'tracks': [57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]}
{'pid': 994002, 'title': 'stu(dying)', 'tracks': [71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118]}
{'pid': 994003, 'title': 'piano', 'tracks': [119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170]}
{'pid': 994004, 'title': 'like', 'tracks': [171, 172, 173, 174, 175, 

In [ ]:
#importing the inputs module
import inputs
importlib.reload(inputs)

<module 'inputs' from '/content/drive/My Drive/workspace/PlaylistCompletion/inputs.py'>

# 3.1 Input vectors



```python
import gensim
import numpy as np
from sklearn.cluster import KMeans
import fasttext
from sklearn.metrics.pairwise import cosine_similarity

path = '/content/drive/MyDrive/workspace/PlaylistCompletion/'

class Corpus:
    """
    iterator for gensim model to train on. returns Name of tracks, albums, artist given tid
    """
    def __init__(self, dataset, vec2get):
        self.dataset = dataset
        if vec2get == "track":
            self.fn = self.dataset.trackName4tid
        elif vec2get == "album":
            self.fn = self.dataset.albumName4tid
        elif vec2get == "artist":
            self.fn = self.dataset.artistName4tid

    def __iter__(self):
        """
        iterator method yields artist name, album name and track name for a particular track_id
        """
        play_gen = self.dataset.playlist_gen()
        for playlist in play_gen:
            for track_id in playlist['tracks']:
                yield self.fn(track_id).split(" ")
            
def word2vectors(dataset):
    """
    trains word2vec models on tracks, albums, artists seperately and use them to get track2vec, album2vec and 
    artist2vec
    """
    tracks = Corpus(dataset, 'track')
    tracksmodel = gensim.models.Word2Vec(sentences=tracks, sg=1, min_count = 1, size=100, window=5, alpha=0.025 , min_alpha=0.0001 ,workers=2)
    gen = dataset.playlist_gen()
    track2vec = {}
    for playlist in gen:
        for track_id in playlist['tracks']:
            if track_id not in track2vec:
                track2vec[track_id] = sum([tracksmodel.wv[word] for word in dataset.trackName4tid(track_id).split(" ")])

                
    artists = Corpus(dataset, 'artist')
    artistsmodel = gensim.models.Word2Vec(sentences=artists, sg=1, min_count=1, size=100, window=5, alpha=0.025, min_alpha=0.0001, workers=2)
    gen = dataset.playlist_gen()
    artist2vec = {}

    for playlist in gen:
        for track_id in playlist['tracks']:
            if track_id not in artist2vec:
                artist2vec[track_id] = sum([artistsmodel.wv[word] for word in dataset.artistName4tid(track_id).split(" ")])


    albums = Corpus(dataset, 'album')
    albumsmodel = gensim.models.Word2Vec(sentences=albums, sg=1, min_count=1, size=100, window=5, alpha=0.025, min_alpha=0.0001, workers=2)
    gen = dataset.playlist_gen()
    album2vec = {}
    for playlist in gen:
        for track_id in playlist['tracks']:
            if track_id not in album2vec:
                album2vec[track_id] = sum([albumsmodel.wv[word] for word in dataset.albumName4tid(track_id).split(" ")])
                

    return track2vec, album2vec, artist2vec


def textfast(file_path):
    """
    Fast text model
    """
    fastmodel = fasttext.train_unsupervised(file_path, model='skipgram', lr=0.1, epoch=5, loss='softmax', ws=5)
    return fastmodel
    

def cluster2rec(track2vec, dataset, n_clusters=500,load=False):
    """
    performs kmeans clustering on playlists and returns fasttext model
    """
    if not load:
        gen = dataset.playlist_gen()
        playlist2vec = {}
        for playlist in gen:
            playlist2vec[playlist['pid']] = [playlist['title'], np.mean([track2vec[track_id] for track_id in playlist['tracks']],axis=0)]
        X = np.array([playlist2vec[pid][1] for pid in playlist2vec])
        kmeans = KMeans(n_clusters=10, random_state=0).fit(X)
        clusters = kmeans.predict(X)
        clstr = iter(clusters)
        for pid in playlist2vec:
            playlist2vec[pid].append(next(clstr))
        clustertext = {c:'' for c in set(clusters)}
        for pid in playlist2vec:
            clustertext[playlist2vec[pid][2]] += (" "+ playlist2vec[pid][0])
        f = open(f"{path}/text.txt", 'w')
        for id in clustertext:
            print(clustertext[id])
            f.write(clustertext[id]+ "\n")
        f.close()
    model = textfast(f"{path}/text.txt")
    
    return model
     
def cosine_sim(a, b):
    """computes cosine similarity between two vectors"""
    dot = np.dot(a,b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    return dot / (norma *normb)

def title2rec(title, model, dataset):
    """
    Given a playlist title returns 300 most similar playlists
    """
    tEmbed = model.get_sentence_vector(title)
    similar = []
    gen = dataset.playlist_gen()
    for playlist in gen:
        similar.append((playlist['pid'], playlist['title'], cosine_sim(model.get_sentence_vector(playlist['title'].strip()), tEmbed)))
    similar.sort(key=lambda x: x[2], reverse=True)
    return similar[:300]
    
    
    

        
    
```



In [ ]:
def pprint(t, n):
  i = 0
  for tid in t:
    if i< n:
      print(t[tid])
      i+= 1
    else:
      break


we get title2vec, album2vec and artist2vec after training 

In [ ]:
#get track2vec , album2vec, artist2vec 
t2v, al2v, ar2v = inputs.word2vectors(d)

In [ ]:
pprint(t2v,2)

[ 0.2999925  -1.0742042   0.22246337  1.2369757   1.6257232   0.13984051
  1.6435018   1.6120446  -1.3600571  -0.18604381  0.15286689 -0.09012099
  0.2586633  -1.5119994   0.19046468  2.312414    1.6210983  -0.73148054
 -1.779562    1.7379537  -1.5581108  -1.8035488  -1.0820193  -0.32789552
 -0.75013995  3.526988    1.5291765   1.7551314   0.7874234   0.6018382
  0.40023556 -0.32034442  0.04809006  0.5151567  -0.58228284 -1.5205154
 -1.2860799   0.29704958  0.9538381   1.5312917  -1.9008076   2.1324143
  1.5323781  -1.3948408   0.2228676  -1.4452574   0.31524318  0.6017426
 -2.646452    1.7631377  -0.00686712  1.9433069   1.328882    1.3939359
  1.3076636  -1.3255936  -0.56110656 -1.350493    0.57006127 -1.8198433
  1.731207    2.195562   -2.312071   -0.38235456 -0.1743545   0.21688244
 -0.9820881  -1.2265277   0.47748798  0.29565576 -1.2434272  -0.19754139
  0.18774317  0.85928774  0.39841393 -1.0605948   1.3512185   0.6186586
 -2.2110522  -0.6653266   0.27433366 -1.4032404  -0.039012

they are 100 dimensional vectors. for each tracks they will be stacked later to create 300 dim vectors

So according to paper, we should stack each of 100dim vectors for track, album and artist to get 300 dim vector. 

# 4. Title2rec

given a playlist name title2rec should return 300 most similar playlists with corresponding similarity scores

In [ ]:
model = inputs.cluster2rec(t2v, d, load=True)

In [ ]:
ls = inputs.title2rec("Country chill", model, d)

title2rec returns 300 most similar playlists pid, name  and corresponding similarity measure.

In [ ]:
ls

[(994423, 'chill playlist', 0.9769992),
 (994407, 'Chillin.', 0.96391124),
 (994404, 'Chillax', 0.96157926),
 (994412, 'chill out ', 0.9569892),
 (994024, 'Playlist', 0.9262064),
 (995517, 'The Ultimate Playlist', 0.889528),
 (994398, 'Welcome Back', 0.8869601),
 (994368, 'quiet time ', 0.8769456),
 (995510, 'Christian Songs', 0.8556213),
 (995516, 'feelin it ', 0.85380286),
 (995499, 'Jan 2015', 0.85121566),
 (995502, 'Awesome music', 0.84844667),
 (994436, 'One Dance', 0.8340094),
 (994022, 'Hip Hop', 0.8232762),
 (994025, 'oth', nan),
 (994100, 'Chill Mix', 0.9226472),
 (994098, 'The Playlist', 0.8813678),
 (994036, 'Mixed Playlist', 0.8811181),
 (994180, 'Just Listen', 0.8719408),
 (994107, 'Hiphop', 0.87111354),
 (994088, 'Tasty Jams', 0.8610036),
 (994017, 'New', 0.8601875),
 (994744, 'feels', 0.8484457),
 (994089, 'Good', 0.82435983),
 (995526, 'The Classics', 0.8207369),
 (995500, 'Morning', 0.82001466),
 (995540, 'Thinking ', 0.81909335),
 (995542, 'June 2015', 0.81245476),
 (

 #  RNN

### rnn.py



```python
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, TimeDistributed
import numpy as np
def get_trackvec(t2v, al2v, ar2v):
    """for each track it concatenates track, album and artist embeddings to get 300dim vector"""
    trackvec = {}
    for tid in t2v:
        trackvec[tid] = np.concatenate((t2v[tid], al2v[tid], ar2v[tid]), axis=None)
    return trackvec

def train_generator(t2v, al2v, ar2v, dataset):
    """
    returns a generator object that returns train and output of dim (n_tracks-1)*300 for each playlist
    """
    gen = dataset.playlist_gen()
    trackvec = get_trackvec(t2v, al2v, ar2v)
    for playlist in gen:
        x_train = np.array([trackvec[tid] for tid in playlist['tracks'][:-1]])
        y_train = np.array([trackvec[tid] for tid in playlist['tracks'][1:]])
        yield x_train, y_train

def rnn():
    """
    compiles and returns rnn model
    """
    model = Sequential()

    model.add(LSTM(128, return_sequences=True, input_shape=(None,300)))
#    model.add(Dropout(0.2))


    model.add(TimeDistributed(Dense(32,activation='relu')))
 #   model.add(Dropout(0.2))

    model.add(TimeDistributed(Dense(1, activation='softmax')))

    opt = tf.keras.optimizers.Adam(lr=0.001)#, decay=1e-6)

    # Compile model
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt
    )
    return model

def recommender(seed_playlist, model):
    """
    Adds tracks to playlists till there are 500 tracks. 
    """
    while len(seed_playlist['tracks']) < 500:
        next_track_id = max([track for track in tracks if track not in seed_playlist['tracks']], key=lambda x: model.get_logits(x))
        seed_playlist['tracks'].append(next_track_id)
    return seed_playlist


```



In [ ]:
import rnn
importlib.reload(rnn)

<module 'rnn' from '/content/drive/My Drive/workspace/PlaylistCompletion/rnn.py'>

Keras has provision that we can also train our model on generator objects in case of large datasets. so, what this generator object does is concatenates track, album and artist embeddings and returns a playlist to train on one by one. 

In [ ]:
#necessary comments are in the modules itself
train_gen = rnn.train_generator(t2v, al2v, ar2v, d)

In [ ]:
rnn_model = rnn.rnn()

In [ ]:
rnn_model.fit(train_gen)

only filling 500 due to colab restrictions on runtime. so we are only getting 500 top playlists yielded by playlist generator. 

In [ ]:
gen = list(d.playlist_gen())[:500]

In [ ]:
# for each of these playlists running the recommender to fill the playlist
for playlist in gen:
  rnn.recommender(playlist,rnn_model)


In [ ]:
#Let's check the length of playlists after running recommender on them. 
for playlist in gen:
  print(len(playlist['tracks']))


500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500
500


I have used fasttext model to implement title2rec. I trained a fasttext model on playlist titles which we already have and when we get a trained model. When we get a new playlist title, title2rec calculates the cosine similarity of new playlist title embedding to all other playlist titles embeddings and returns top 300 playlist titles. 

As for the RNN model, I used single model to train and fill playlists. I did not implement ensemble of RNNs. 